# Phase 1

In [ ]:
!pip install faiss-cpu sentence-transformers

In [ ]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# =========================
# CONFIG
# =========================
DATA_PATH = "/content/rag_chunks_engineered_v2.jsonl"
INDEX_PATH = "faiss_index_v2.bin"
METADATA_PATH = "metadata_v2.json"

CHUNK_SIZE = 900
CHUNK_OVERLAP = 200

# =========================
# LOAD MODEL (MXBAI LARGE)
# =========================
e_model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

# =========================
# CHUNKING FUNCTION
# =========================
def chunk_text(text, size=900, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + size, len(text))
        chunks.append(text[start:end])
        start += size - overlap
    return chunks

documents = []
metadata = []

# =========================
# LOAD + CHUNK DATA
# =========================
with open(DATA_PATH, "r") as f:
    for line in f:
        obj = json.loads(line)

        if obj["doc_type"] != "table_of_cover":
            continue

        full_text = obj["text"]
        plan_name = obj.get("plan_name", "Unknown Plan")
        insurer = obj.get("insurer", "Unknown Insurer")
        doc_id = obj.get("doc_id")

        chunks = chunk_text(full_text, CHUNK_SIZE, CHUNK_OVERLAP)

        for i, chunk in enumerate(chunks):

            enriched_chunk = (
                f"[Insurer: {insurer}]\n"
                f"[Plan: {plan_name}]\n"
                f"[Chunk: {i}]\n\n"
                f"{chunk}"
            )

            documents.append(enriched_chunk)

            metadata.append({
                "doc_id": doc_id,
                "insurer": insurer,
                "plan_name": plan_name,
                "chunk_id": i,
                "chunk_text": chunk,
                "full_text": full_text
            })

print(f"Total chunks indexed: {len(documents)}")

# =========================
# CREATE EMBEDDINGS
# =========================
embeddings = e_model.encode(
    documents,
    batch_size=16,
    show_progress_bar=True,
    normalize_embeddings=True
)

embeddings = np.array(embeddings).astype("float32")

# =========================
# BUILD FAISS INDEX (Cosine)
# =========================
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

faiss.write_index(index, INDEX_PATH)

with open(METADATA_PATH, "w") as f:
    json.dump(metadata, f)

print("🔥 FAISS v2 index built successfully.")

# ==========================================================
# SEARCH FUNCTION (Parent-Plan Aggregation)
# ==========================================================
def search(query, k=5):
    # MXBAI works better with query prefix
    prefixed_query = "Represent this sentence for searching relevant passages: " + query

    query_embedding = e_model.encode(
        [prefixed_query],
        normalize_embeddings=True
    )

    query_embedding = np.array(query_embedding).astype("float32")

    scores, indices = index.search(query_embedding, k)

    results = []
    seen_plans = set()

    for idx in indices[0]:
        plan_name = metadata[idx]["plan_name"]

        if plan_name not in seen_plans:
            seen_plans.add(plan_name)

            results.append({
                "plan_name": plan_name,
                "insurer": metadata[idx]["insurer"],
                "matched_chunk": metadata[idx]["chunk_text"],
                "full_text": metadata[idx]["full_text"]
            })

    return results


# =========================
# TEST QUERY
# =========================
query = """
Coverage for cardiac treatment, frequent hospital visits,
consultant fees, medication support,
and minimal excess for inpatient admission.
"""

results = search(query)

for r in results:
    print("Plan:", r["plan_name"])
    print("Insurer:", r["insurer"])
    print("Matched Section Preview:")
    print(r["matched_chunk"][:500])
    print("=" * 80)


# Phase 2

In [ ]:
import json
import numpy as np
import faiss
from collections import defaultdict
from sentence_transformers import SentenceTransformer

# ----------------------------
# CONFIG
# ----------------------------

INDEX_PATH = "faiss_index_v2.bin"
METADATA_PATH = "metadata_v2.json"
TOP_K = 3

# ----------------------------
# LOAD MODEL + INDEX
# ----------------------------

print("Loading embedding model...")
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

print("Loading FAISS index...")
index = faiss.read_index(INDEX_PATH)

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

# ----------------------------
# SEARCH FUNCTION (Chunk → Plan Aggregation)
# ----------------------------

def search(query, k=20):  # retrieve more chunks first
    prefixed_query = "Represent this sentence for searching relevant passages: " + query

    query_embedding = model.encode(
        [prefixed_query],
        normalize_embeddings=True
    )

    query_embedding = np.array(query_embedding).astype("float32")

    scores, indices = index.search(query_embedding, k)

    # Aggregate scores per plan
    plan_scores = defaultdict(list)

    for score, idx in zip(scores[0], indices[0]):
        plan_id = metadata[idx]["doc_id"]
        plan_scores[plan_id].append(score)

    # Average score per plan
    aggregated = []
    for plan_id, score_list in plan_scores.items():
        avg_score = sum(score_list) / len(score_list)
        aggregated.append((plan_id, avg_score))

    # Sort descending
    aggregated.sort(key=lambda x: x[1], reverse=True)

    # Return top unique plans
    return [plan_id for plan_id, _ in aggregated[:TOP_K]]

# ----------------------------
# AUTO DOC_ID RESOLUTION
# ----------------------------

def get_doc_id(plan_keyword):
    matches = [
        m["doc_id"]
        for m in metadata
        if plan_keyword.lower() in m["plan_name"].lower()
    ]
    if not matches:
        raise ValueError(f"No doc_id found for keyword: {plan_keyword}")
    return matches[0]

# ----------------------------
# BUILD EVALUATION SET
# ----------------------------

evaluation_set = [
    {
        "query": "Does Horizon 4 cover inpatient consultant fees?",
        "expected_doc_id": get_doc_id("Horizon 4")
    },
    {
        "query": "Which plan has a €300 excess for semi-private room admission?",
        "expected_doc_id": get_doc_id("300")
    },
    {
        "query": "How many days of psychiatric treatment are covered under Plan A?",
        "expected_doc_id": get_doc_id("Plan A")
    },
    {
        "query": "Are inpatient scans fully covered under Health Plan 26.1?",
        "expected_doc_id": get_doc_id("26.1")
    }
]

# ----------------------------
# METRIC COMPUTATION
# ----------------------------

correct_top1 = 0
correct_topk = 0
reciprocal_ranks = []

print("\nRunning evaluation...\n")

for item in evaluation_set:
    query = item["query"]
    expected = item["expected_doc_id"]

    returned_ids = search(query)

    print("Query:", query)
    print("Expected:", expected)
    print("Returned:", returned_ids)
    print("-" * 60)

    if returned_ids[0] == expected:
        correct_top1 += 1

    if expected in returned_ids:
        correct_topk += 1
        rank = returned_ids.index(expected) + 1
        reciprocal_ranks.append(1.0 / rank)
    else:
        reciprocal_ranks.append(0.0)

# ----------------------------
# FINAL METRICS
# ----------------------------

total = len(evaluation_set)

top1_accuracy = correct_top1 / total
topk_recall = correct_topk / total
mrr = sum(reciprocal_ranks) / total

print("\n===== FINAL METRICS =====")
print(f"Top-1 Accuracy: {top1_accuracy:.3f}")
print(f"Top-{TOP_K} Recall: {topk_recall:.3f}")
print(f"MRR: {mrr:.3f}")


# Phase 3

In [ ]:
! pip install rank-bm25

In [ ]:
import json
import numpy as np
import re
from collections import defaultdict
from rank_bm25 import BM25Okapi

# ----------------------------
# CONFIG
# ----------------------------

METADATA_PATH = "metadata_v2.json"
TOP_K = 3

# ----------------------------
# LOAD CHUNKED METADATA
# ----------------------------

print("Loading chunked metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

print(f"Using {len(metadata)} chunks")

# ----------------------------
# TOKENIZATION
# ----------------------------

def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9€]+", " ", text)
    return text.split()

corpus = [m["chunk_text"] for m in metadata]
tokenized_corpus = [tokenize(doc) for doc in corpus]

print("Building BM25 chunk-level index...")
bm25 = BM25Okapi(tokenized_corpus)

# ----------------------------
# SEARCH (Chunk → Plan Aggregation)
# ----------------------------

def search_bm25(query, k=20):
    tokenized_query = tokenize(query)
    scores = bm25.get_scores(tokenized_query)

    # Aggregate scores per plan
    plan_scores = defaultdict(list)

    for idx, score in enumerate(scores):
        plan_id = metadata[idx]["doc_id"]
        plan_scores[plan_id].append(score)

    aggregated = []
    for plan_id, score_list in plan_scores.items():
        avg_score = sum(score_list) / len(score_list)
        aggregated.append((plan_id, avg_score))

    aggregated.sort(key=lambda x: x[1], reverse=True)

    return [plan_id for plan_id, _ in aggregated[:TOP_K]]

# ----------------------------
# DOC ID RESOLUTION
# ----------------------------

def get_doc_id(plan_keyword):
    matches = [
        m["doc_id"]
        for m in metadata
        if plan_keyword.lower() in m["plan_name"].lower()
    ]
    if not matches:
        raise ValueError(f"No doc_id found for keyword: {plan_keyword}")
    return matches[0]

# ----------------------------
# EVALUATION SET
# ----------------------------

evaluation_set = [
    {
        "query": "Does Horizon 4 cover inpatient consultant fees?",
        "expected_doc_id": get_doc_id("Horizon 4")
    },
    {
        "query": "Which plan has a €300 excess for semi-private room admission?",
        "expected_doc_id": get_doc_id("300")
    },
    {
        "query": "How many days of psychiatric treatment are covered under Plan A?",
        "expected_doc_id": get_doc_id("Plan A")
    },
    {
        "query": "Are inpatient scans fully covered under Health Plan 26.1?",
        "expected_doc_id": get_doc_id("26.1")
    }
]

# ----------------------------
# METRICS
# ----------------------------

correct_top1 = 0
correct_topk = 0
reciprocal_ranks = []

print("\nRunning BM25 v2 evaluation...\n")

for item in evaluation_set:
    query = item["query"]
    expected = item["expected_doc_id"]

    returned_ids = search_bm25(query)

    print("Query:", query)
    print("Returned:", returned_ids)
    print("-" * 60)

    if returned_ids[0] == expected:
        correct_top1 += 1

    if expected in returned_ids:
        correct_topk += 1
        rank = returned_ids.index(expected) + 1
        reciprocal_ranks.append(1.0 / rank)
    else:
        reciprocal_ranks.append(0.0)

total = len(evaluation_set)

print("\n===== BM25 v2 =====")
print("Top-1:", correct_top1 / total)
print("Top-3:", correct_topk / total)
print("MRR:", sum(reciprocal_ranks) / total)


# Phase 4

In [ ]:
import json
import numpy as np
import faiss
import re
from collections import defaultdict
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer

# ----------------------------
# CONFIG
# ----------------------------

INDEX_PATH = "faiss_index_v2.bin"
METADATA_PATH = "metadata_v2.json"
TOP_K = 3
ALPHA = 0.6
BETA = 0.4

# ----------------------------
# LOAD DATA
# ----------------------------

print("Loading MXBAI model...")
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

print("Loading FAISS index...")
index = faiss.read_index(INDEX_PATH)

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

print(f"Using {len(metadata)} chunks")

# ----------------------------
# TOKENIZATION
# ----------------------------

def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9€]+", " ", text)
    return text.split()

corpus = [m["chunk_text"] for m in metadata]
tokenized_corpus = [tokenize(doc) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

# ----------------------------
# NORMALIZATION
# ----------------------------

def normalize(scores_dict):
    scores = np.array(list(scores_dict.values()))
    min_s = scores.min()
    max_s = scores.max()

    normalized = {}
    for k, v in scores_dict.items():
        if max_s - min_s == 0:
            normalized[k] = 0.0
        else:
            normalized[k] = (v - min_s) / (max_s - min_s)
    return normalized

# ----------------------------
# HYBRID SEARCH
# ----------------------------

def hybrid_search(query, k=TOP_K):

    # -------- DENSE --------
    prefixed_query = "Represent this sentence for searching relevant passages: " + query

    query_embedding = model.encode(
        [prefixed_query],
        normalize_embeddings=True
    )

    query_embedding = np.array(query_embedding).astype("float32")

    dense_scores, dense_indices = index.search(query_embedding, 50)

    dense_plan_scores = defaultdict(list)

    for score, idx in zip(dense_scores[0], dense_indices[0]):
        plan_id = metadata[idx]["doc_id"]
        dense_plan_scores[plan_id].append(score)

    dense_plan_scores = {
        plan: sum(scores)/len(scores)
        for plan, scores in dense_plan_scores.items()
    }

    # -------- BM25 --------
    tokenized_query = tokenize(query)
    bm25_scores_all = bm25.get_scores(tokenized_query)

    bm25_plan_scores = defaultdict(list)

    for idx, score in enumerate(bm25_scores_all):
        plan_id = metadata[idx]["doc_id"]
        bm25_plan_scores[plan_id].append(score)

    bm25_plan_scores = {
        plan: sum(scores)/len(scores)
        for plan, scores in bm25_plan_scores.items()
    }

    # -------- NORMALIZE --------
    dense_norm = normalize(dense_plan_scores)
    bm25_norm = normalize(bm25_plan_scores)

    # -------- FUSION --------
    final_scores = {}

    all_plans = set(dense_norm.keys()) | set(bm25_norm.keys())

    for plan in all_plans:
        d = dense_norm.get(plan, 0)
        b = bm25_norm.get(plan, 0)
        final_scores[plan] = ALPHA * d + BETA * b

    ranked = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)

    return [plan_id for plan_id, _ in ranked[:k]]

# ----------------------------
# DOC ID RESOLUTION
# ----------------------------

def get_doc_id(plan_keyword):
    matches = [
        m["doc_id"]
        for m in metadata
        if plan_keyword.lower() in m["plan_name"].lower()
    ]
    if not matches:
        raise ValueError(f"No doc_id found for keyword: {plan_keyword}")
    return matches[0]

# ----------------------------
# EVALUATION SET
# ----------------------------

evaluation_set = [
    {
        "query": "Does Horizon 4 cover inpatient consultant fees?",
        "expected_doc_id": get_doc_id("Horizon 4")
    },
    {
        "query": "Which plan has a €300 excess for semi-private room admission?",
        "expected_doc_id": get_doc_id("300")
    },
    {
        "query": "How many days of psychiatric treatment are covered under Plan A?",
        "expected_doc_id": get_doc_id("Plan A")
    },
    {
        "query": "Are inpatient scans fully covered under Health Plan 26.1?",
        "expected_doc_id": get_doc_id("26.1")
    }
]

# ----------------------------
# METRICS
# ----------------------------

correct_top1 = 0
correct_topk = 0
reciprocal_ranks = []

print("\nRunning Hybrid v2 evaluation...\n")

for item in evaluation_set:
    query = item["query"]
    expected = item["expected_doc_id"]

    returned_ids = hybrid_search(query)

    print("Query:", query)
    print("Expected:", expected)
    print("Returned:", returned_ids)
    print("-" * 60)

    if returned_ids[0] == expected:
        correct_top1 += 1

    if expected in returned_ids:
        correct_topk += 1
        rank = returned_ids.index(expected) + 1
        reciprocal_ranks.append(1.0 / rank)
    else:
        reciprocal_ranks.append(0.0)

total = len(evaluation_set)

print("\n===== HYBRID v2 RESULTS =====")
print("Top-1:", correct_top1 / total)
print("Top-3:", correct_topk / total)
print("MRR:", sum(reciprocal_ranks) / total)


# Phase 5

In [ ]:
import json
import re
import numpy as np
import faiss
from collections import defaultdict
from sentence_transformers import SentenceTransformer

# ======================================================
# CONFIG
# ======================================================

INDEX_PATH = "faiss_index_v2.bin"
METADATA_PATH = "metadata_v2.json"

BETA = 0.6           # Risk weight
FINAL_TOP_K = 3

# ======================================================
# LOAD MODEL + INDEX
# ======================================================

print("Loading MXBAI model...")
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

print("Loading FAISS index...")
index = faiss.read_index(INDEX_PATH)

print("Loading metadata...")
with open(METADATA_PATH, "r") as f:
    metadata = json.load(f)

plan_ids = list(set(m["doc_id"] for m in metadata))
print(f"Loaded {len(plan_ids)} unique plans")

# ======================================================
# DENSE RELEVANCE (Chunk → Plan)
# ======================================================

def compute_dense_scores(query):

    prefixed_query = "Represent this sentence for searching relevant passages: " + query

    q_emb = model.encode([prefixed_query], normalize_embeddings=True)
    q_emb = np.array(q_emb).astype("float32")

    dense_scores, dense_indices = index.search(q_emb, 50)

    dense_plan_scores = defaultdict(list)

    for score, idx in zip(dense_scores[0], dense_indices[0]):
        plan_id = metadata[idx]["doc_id"]
        dense_plan_scores[plan_id].append(score)

    dense_plan_scores = {
        plan: np.mean(scores)
        for plan, scores in dense_plan_scores.items()
    }

    # Normalize 0–1
    if dense_plan_scores:
        vals = np.array(list(dense_plan_scores.values()))
        min_v, max_v = vals.min(), vals.max()

        dense_plan_scores = {
            k: (v - min_v) / (max_v - min_v + 1e-8)
            for k, v in dense_plan_scores.items()
        }

    return dense_plan_scores

# ======================================================
# DENSE RETRIEVAL HELPER (Plan-Specific)
# ======================================================

def retrieve_plan_chunks(plan_id, query, top_n=5):

    prefixed_query = "Represent this sentence for searching relevant passages: " + query

    q_emb = model.encode([prefixed_query], normalize_embeddings=True)
    q_emb = np.array(q_emb).astype("float32")

    scores, indices = index.search(q_emb, 50)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        if metadata[idx]["doc_id"] == plan_id:
            results.append((score, metadata[idx]["chunk_text"]))
            if len(results) >= top_n:
                break

    return results

# ======================================================
# DISEASE RULES
# ======================================================

DISEASE_RULES = {
    "heart_disease": {
        "keywords": ["cardiac", "angioplasty", "stent"],
        "weight": 8,
        "requires_high_tech": True
    },
    "diabetes": {
        "keywords": ["diabetes", "insulin"],
        "weight": 5
    },
    "cancer": {
        "keywords": ["oncology", "chemotherapy", "radiotherapy"],
        "weight": 10,
        "requires_high_tech": True
    },
    "psychiatric_disorder": {
        "keywords": ["psychiatric", "mental health"],
        "weight": 7,
        "requires_psych_days": True
    },
    "neurological_disorder": {
        "keywords": ["neurology", "mri", "ct scan"],
        "weight": 8,
        "requires_high_tech": True
    },
    "pregnancy": {
        "keywords": ["maternity", "obstetric"],
        "weight": 8
    }
}

# ======================================================
# STRUCTURED FEATURE EXTRACTION
# ======================================================

def merge_plan_text(plan_id):
    return " ".join(
        m["chunk_text"].lower()
        for m in metadata
        if m["doc_id"] == plan_id
    )

def extract_features(text):

    features = {}

    features["full_inpatient"] = (
        "inpatient consultant fees" in text and "fully covered" in text
    )

    excess_match = re.search(r"€\s?(\d+)\s+excess", text)
    features["excess"] = int(excess_match.group(1)) if excess_match else 0

    copay_match = re.search(r"€\s?(\d+)[^\n]{0,50}cardiac", text)
    features["cardiac_copay"] = int(copay_match.group(1)) if copay_match else 0

    psych_days = re.findall(r"(\d+)\s+days", text)
    features["psychiatric_days"] = max([int(x) for x in psych_days], default=0)

    features["high_tech_available"] = (
        "high-tech hospital" in text and "not covered" not in text
    )

    return features

# ======================================================
# DENSE-GUIDED DISEASE SCORING
# ======================================================

def dense_disease_score(plan_id, condition):

    rules = DISEASE_RULES.get(condition)
    if not rules:
        return 0

    query = f"Coverage details for {condition.replace('_',' ')} including hospital and treatment"

    chunks = retrieve_plan_chunks(plan_id, query)

    score = 0

    for _, chunk in chunks:
        chunk = chunk.lower()

        for kw in rules["keywords"]:
            if kw in chunk:
                score += rules["weight"]

        if rules.get("requires_high_tech"):
            if "high-tech" in chunk and "not covered" not in chunk:
                score += 6
            elif "not covered" in chunk:
                score -= 8

        if rules.get("requires_psych_days"):
            days = re.findall(r"(\d+)\s+days", chunk)
            if days:
                score += min(int(max(days)), 150) / 20

    return score

# ======================================================
# FINAL RISK SCORING
# ======================================================

def compute_risk_score(plan_id, profile):

    text = merge_plan_text(plan_id)
    f = extract_features(text)

    score = 0
    breakdown = {}

    inpatient = 8 if f["full_inpatient"] else -8
    score += inpatient
    breakdown["inpatient"] = inpatient

    excess_penalty = f["excess"] / 60
    score -= excess_penalty
    breakdown["excess_penalty"] = -round(excess_penalty, 2)

    cardiac_penalty = 0
    if "heart_disease" in profile["chronic_conditions"]:
        cardiac_penalty = f["cardiac_copay"] / 40
        score -= cardiac_penalty
    breakdown["cardiac_penalty"] = -round(cardiac_penalty, 2)

    disease_total = 0
    for condition in profile["chronic_conditions"]:
        disease_total += dense_disease_score(plan_id, condition)

    score += disease_total
    breakdown["dense_disease_score"] = round(disease_total, 2)

    return score, breakdown

# ======================================================
# MULTI-SCENARIO EVALUATION
# ======================================================

SCENARIOS = {
    "Cardiac + Diabetes": {
        "profile": {
            "age": 55,
            "chronic_conditions": ["heart_disease", "diabetes"]
        },
        "query": """
        Cardiac procedures, insulin support,
        frequent inpatient admission,
        consultant coverage, high-tech hospital.
        """
    },

    "Cancer + Neurological": {
        "profile": {
            "age": 60,
            "chronic_conditions": ["cancer", "neurological_disorder"]
        },
        "query": """
        Oncology treatment, chemotherapy,
        MRI scans, neurological admission,
        high-tech hospital access.
        """
    },

    "Psychiatric + Diabetes": {
        "profile": {
            "age": 40,
            "chronic_conditions": ["psychiatric_disorder", "diabetes"]
        },
        "query": """
        Psychiatric admission days,
        mental health coverage,
        diabetes management.
        """
    },

    "Pregnancy Case": {
        "profile": {
            "age": 30,
            "chronic_conditions": ["pregnancy"]
        },
        "query": """
        Maternity cover,
        obstetric services,
        hospital delivery,
        minimal excess.
        """
    }
}

print("\n====================================")
print("AUTOMATED MULTI-SCENARIO EVALUATION (DENSE ONLY)")
print("====================================")

for scenario_name, scenario_data in SCENARIOS.items():

    print(f"\n==============================")
    print(f"Scenario: {scenario_name}")
    print("==============================")

    patient_profile = scenario_data["profile"]
    query = scenario_data["query"]

    dense_scores = compute_dense_scores(query)

    results = []

    for plan_id in plan_ids:

        risk_score, breakdown = compute_risk_score(plan_id, patient_profile)

        final_score = (BETA * risk_score) + dense_scores.get(plan_id, 0)

        results.append((plan_id, final_score, risk_score,
                        dense_scores.get(plan_id, 0), breakdown))

    results.sort(key=lambda x: x[1], reverse=True)

    for rank, (doc_id, final_score, risk_score,
               dense_score, breakdown) in enumerate(results[:FINAL_TOP_K], 1):

        print(f"\n{rank}. {doc_id}")
        print(f"   Final Score: {round(final_score,2)}")
        print(f"   Risk Score: {round(risk_score,2)}")
        print(f"   Dense Score: {round(dense_score,3)}")

        print("   Breakdown:")
        for k, v in breakdown.items():
            print(f"     {k}: {v}")

    margin = results[0][1] - results[1][1]

    print("\nConfidence Margin:", round(margin, 3))


In [ ]:
# ======================================================
# ADVANCED MULTI-SCENARIO EVALUATION
# ======================================================

SCENARIOS = {

    "Cardiac + Diabetes": {
        "profile": {
            "age": 55,
            "chronic_conditions": ["heart_disease", "diabetes"]
        },
        "query": """
        Cardiac procedures, insulin support,
        frequent inpatient admission,
        consultant coverage, high-tech hospital.
        """
    },

    "Cancer + Neurological": {
        "profile": {
            "age": 60,
            "chronic_conditions": ["cancer", "neurological_disorder"]
        },
        "query": """
        Oncology treatment, chemotherapy,
        MRI scans, neurological admission,
        high-tech hospital access.
        """
    },

    "Psychiatric + Diabetes": {
        "profile": {
            "age": 40,
            "chronic_conditions": ["psychiatric_disorder", "diabetes"]
        },
        "query": """
        Psychiatric admission days,
        mental health coverage,
        diabetes management.
        """
    },

    "Pregnancy Case": {
        "profile": {
            "age": 30,
            "chronic_conditions": ["pregnancy"]
        },
        "query": """
        Maternity cover,
        obstetric services,
        hospital delivery,
        minimal excess.
        """
    },

    "High-Risk Elderly Multi-Morbidity": {
        "profile": {
            "age": 72,
            "chronic_conditions": ["heart_disease", "cancer", "neurological_disorder"]
        },
        "query": """
        Cardiac procedures, chemotherapy,
        MRI scans, frequent hospital admissions,
        high-tech hospital access.
        """
    },

    "Long-Term Psychiatric Intensive": {
        "profile": {
            "age": 42,
            "chronic_conditions": ["psychiatric_disorder"]
        },
        "query": """
        Extended psychiatric inpatient treatment,
        high number of covered days,
        strong mental health support.
        """
    }
}

print("\n====================================")
print("ADVANCED MULTI-SCENARIO EVALUATION (DENSE ONLY)")
print("====================================")

plan_win_counter = defaultdict(int)
plan_rank_sum = defaultdict(int)
scenario_margins = []

for scenario_name, scenario_data in SCENARIOS.items():

    print("\n==============================")
    print(f"Scenario: {scenario_name}")
    print("==============================")

    patient_profile = scenario_data["profile"]
    query = scenario_data["query"]

    dense_scores = compute_dense_scores(query)

    results = []

    for plan_id in plan_ids:

        risk_score, breakdown = compute_risk_score(plan_id, patient_profile)

        final_score = (BETA * risk_score) + dense_scores.get(plan_id, 0)

        results.append((plan_id, final_score, risk_score,
                        dense_scores.get(plan_id, 0), breakdown))

    results.sort(key=lambda x: x[1], reverse=True)

    # Track winner
    winner = results[0][0]
    plan_win_counter[winner] += 1

    # Track average ranks
    for rank, (doc_id, *_ ) in enumerate(results, 1):
        plan_rank_sum[doc_id] += rank

    # Print top results
    for rank, (doc_id, final_score, risk_score,
               dense_score, breakdown) in enumerate(results[:FINAL_TOP_K], 1):

        print(f"\n{rank}. {doc_id}")
        print(f"   Final Score: {round(final_score,2)}")
        print(f"   Risk Score: {round(risk_score,2)}")
        print(f"   Dense Score: {round(dense_score,3)}")

        print("   Breakdown:")
        for k, v in breakdown.items():
            print(f"     {k}: {v}")

    margin = results[0][1] - results[1][1]
    scenario_margins.append(margin)

    print("\nConfidence Margin:", round(margin, 3))

    if margin < 1:
        print("Interpretation: Close competition")
    elif margin < 3:
        print("Interpretation: Moderate separation")
    else:
        print("Interpretation: Strong winner")

# ======================================================
# GLOBAL ANALYTICS
# ======================================================

print("\n====================================")
print("GLOBAL ANALYTICS")
print("====================================")

print("\nPlan Win Frequency:")
for plan, count in sorted(plan_win_counter.items(), key=lambda x: x[1], reverse=True):
    print(f"{plan}: {count} wins")

print("\nAverage Rank Per Plan:")
num_scenarios = len(SCENARIOS)

avg_ranks = {
    plan: plan_rank_sum[plan] / num_scenarios
    for plan in plan_ids
}

for plan, avg_rank in sorted(avg_ranks.items(), key=lambda x: x[1]):
    print(f"{plan}: Avg Rank = {round(avg_rank,2)}")

print("\nAverage Confidence Margin Across Scenarios:",
      round(np.mean(scenario_margins), 3))

print("Margin Std Dev:",
      round(np.std(scenario_margins), 3))


# Phase 6

In [ ]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# =====================================================
# CONFIG
# =====================================================

MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
TOP_K_PLANS = 3
MAX_NEW_TOKENS = 180

# =====================================================
# STEP 1 — PREPARE RANKED RESULTS
# =====================================================

dense_scores = compute_dense_scores(query)

ranked_results = []

for plan_id in plan_ids:
    risk_score, breakdown = compute_risk_score(plan_id, patient_profile)
    final_score = (BETA * risk_score) + dense_scores.get(plan_id, 0)

    ranked_results.append({
        "doc_id": plan_id,
        "final_score": final_score,
        "risk_score": risk_score,
        "dense_score": dense_scores.get(plan_id, 0),
        "breakdown": breakdown
    })

ranked_results.sort(key=lambda x: x["final_score"], reverse=True)

top_plans = ranked_results[:TOP_K_PLANS]

# =====================================================
# STEP 2 — BUILD STRUCTURED + DENSE EVIDENCE
# =====================================================

def build_dense_evidence(plan_id, profile):

    text = merge_plan_text(plan_id)
    features = extract_features(text)

    evidence = {
        "full_inpatient": features["full_inpatient"],
        "excess": features["excess"],
        "cardiac_copay": features["cardiac_copay"],
        "high_tech_available": features["high_tech_available"],
        "psychiatric_days": features["psychiatric_days"]
    }

    # Dense chunk evidence per condition
    retrieved_chunks = {}

    for condition in profile["chronic_conditions"]:
        condition_query = f"Coverage details for {condition.replace('_',' ')}"
        chunks = retrieve_plan_chunks(plan_id, condition_query, top_n=2)

        retrieved_chunks[condition] = [
            chunk_text[:200] for _, chunk_text in chunks
        ]

    evidence["retrieved_chunks"] = retrieved_chunks

    return evidence

# =====================================================
# STEP 3 — LOAD LLM (SAFE FP16 VERSION)
# =====================================================

print("\nLoading Explanation LLM...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

llm_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

llm_model.eval()

print("LLM loaded.")

# =====================================================
# STEP 4 — EXPLANATION GENERATOR
# =====================================================

def generate_explanation(rank_data, next_plan=None):

    evidence = build_dense_evidence(rank_data["doc_id"], patient_profile)

    comparison_text = ""
    if next_plan:
        score_diff = round(
            rank_data["final_score"] - next_plan["final_score"], 2
        )

        comparison_text = f"""
Comparison With Next Ranked Plan:
Next Plan ID: {next_plan['doc_id']}
Score Difference: {score_diff}
Next Plan Risk Score: {round(next_plan['risk_score'],2)}
Next Plan Dense Score: {round(next_plan['dense_score'],3)}
"""

    prompt = f"""
You are a medical insurance ranking explanation engine.

IMPORTANT RULES:
- Use ONLY the structured evidence provided.
- Do NOT invent benefits.
- Do NOT assume coverage beyond evidence.
- Explain ranking strictly according to the scoring model.
- Mention inpatient impact, excess penalty, disease alignment, and dense relevance.
- Clarify that ranking is model-based.
- Maximum 170 words.

PATIENT PROFILE:
{json.dumps(patient_profile)}

PLAN ID: {rank_data['doc_id']}
FINAL SCORE: {round(rank_data['final_score'],2)}
RISK SCORE: {round(rank_data['risk_score'],2)}
DENSE SCORE: {round(rank_data['dense_score'],3)}

SCORE BREAKDOWN:
{json.dumps(rank_data['breakdown'], indent=2)}

EVIDENCE:
{json.dumps(evidence, indent=2)}

{comparison_text}

Explain why this plan ranked at this position.
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(llm_model.device)

    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove prompt echo if present
    if decoded.startswith(prompt):
        decoded = decoded[len(prompt):]

    return decoded.strip()

# =====================================================
# STEP 5 — RUN EXPLANATIONS
# =====================================================

print("\n====================================")
print("DENSE-AWARE COMPARATIVE EXPLANATIONS")
print("====================================")

for i, plan_data in enumerate(top_plans):

    next_plan = top_plans[i+1] if i+1 < len(top_plans) else None

    explanation = generate_explanation(plan_data, next_plan)

    print("\n====================================")
    print("PLAN:", plan_data["doc_id"])
    print("====================================")
    print(explanation)
